<a id='top'></a>

# checkName

checkName uses a database lookup to return either
SMILES or IUPAC names of salts.

# Contents

[script that takes in existing work Coco has done](#scriptgrab)

In [47]:
def checkName(user_query):
    """
    checkName uses a database lookup to return either SMILES or IUPAC 
    names of salts given either one of those are provided as inputs.
    Default behavior is to return the SMILES encoding of a salt given
    the salt name as input.
    
    Parameters
    ------------------
    user_query : str
        string that will be used to query the database.
        
    Returns
    ------------------
    output: str
        either the name of the salt, cation, or anion; or SMILES of the
        salt, cation, or anion (SMILES for the salt are written as the 
        cation and ion SMILES strings separated by a comma)
    """
    ###function imports
    import os
    import pandas as pd
    import numpy as np
    
    ###Check to see that the database is present
    if os.path.isfile('../salty/data/saltInfo.csv'):
        print('database found')
    else:
        print('database file missing... exiting')
        quit()
    df = pd.read_csv('../salty/data/saltInfo.csv').astype(str)

    try:
        target_lookup = df.loc[(df == user_query).any(axis=1),:]
        input_type = df.loc[:,(df == user_query).any(axis=0)].columns.values
        target_column_index = df.columns.get_loc(input_type[0])
    except:
        print("query not found")
        return 0

    #row_index pairs 1-4, 2-5, and 3-6
    if target_column_index == 1 or target_column_index == 2 or target_column_index == 3:
        print("user has queried with a SMILES structure")
        target = target_lookup.iloc[0][target_column_index+3]
    else:
        print("user has queried with a name")
        target = target_lookup.iloc[0][target_column_index-3]
    print("your query has returned %s" % target)
    return target

In [48]:
checkName("1-butyl-3-methylimidazolium ,trifluoroacetate")

database found
query not found


0

In [45]:
df = pd.read_csv('../salty/data/saltInfo.csv').astype(str)
user_query="1-butyl-3-methylimidazolium ,trifluoromethanesulfonate"
target_lookup = df.loc[(df == user_query).any(axis=1),:]
input_type = df.loc[:,(df == user_query).any(axis=0)].columns.values
target_column_index = df.columns.get_loc(input_type[0])

#row_index pairs 1-4, 2-5, and 3-6
if target_column_index == 1 or target_column_index == 2 or target_column_index == 3:
    print("user has queried with a SMILES structure")
    target = target_lookup.iloc[0][target_column_index+3]
else:
    print("user has queried with a name")
    target = target_lookup.iloc[0][target_column_index-3]
print("your query has returned %s" % target)

print(target)

user has queried with a name
your query has returned CCCCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]
CCCCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]


<a id='scriptgrab'></a>

# Script that reads in Coco's salt names

[back to top](#top)

In [25]:
###READ SMILES INTO PYCHEM AND CREATE DATAFRAME
import pandas as pd
salts=pd.read_csv('../salty/data/Viscosity.csv',delimiter=',')

# create names and smiles of entire salt
ILThermoName=pd.Series(salts['SMILE_cation'].str.cat(salts['SMILE_anion'], sep=','))
saltName=pd.Series(salts['NAME_CAT'].str.cat(salts['NAME_ANI'], sep=','))
old = pd.DataFrame()
new = pd.DataFrame()

for a, b in enumerate(salts['NAME_CAT']):   
    #create empty arrays
    catSMILES=[]
    anSMILES=[]
    nameSMILES=[]
    nameCat=[]
    nameAn=[]
    name=[]
    
    #append the things we want
    catSMILES.append(salts['SMILE_cation'][a])
    anSMILES.append(salts['SMILE_anion'][a])
    nameSMILES.append(ILThermoName[a])
    nameCat.append(salts['NAME_CAT'][a])
    nameAn.append(salts['NAME_ANI'][a])
    name.append(saltName[a])
    
    #turn into dataframes
    dfcatSMILES = pd.DataFrame(catSMILES)
    dfanSMILES = pd.DataFrame(anSMILES)
    dfnameSMILES = pd.DataFrame(nameSMILES)
    dfnameCat = pd.DataFrame(nameCat)
    dfnameAn = pd.DataFrame(nameAn)
    dfname = pd.DataFrame(name)
    
    #rename headers
    dfcatSMILES.rename(columns=lambda x: 'cation_SMILES', inplace=True) #rename headers (cation or anion)
    dfanSMILES.rename(columns=lambda x: 'anion_SMILES', inplace=True)
    dfnameSMILES.rename(columns=lambda x: 'salt_SMILES', inplace=True)
    dfnameCat.rename(columns=lambda x: 'cation_name', inplace=True)
    dfnameAn.rename(columns=lambda x: 'anion_name', inplace=True)
    dfname.rename(columns=lambda x: 'salt_name', inplace=True)         
    
    #concatonate to master df
    df = pd.concat([dfcatSMILES, dfanSMILES, dfnameSMILES, dfnameCat, dfnameAn, dfname], axis=1) #concatonate cationic and anionic descriptor df's
    if a != 0: #grow the dataframe
        new = pd.concat([df, old], axis=0)
    old = new
    

In [31]:
df = old
df.drop_duplicates(inplace=True)
df.reindex
df.to_csv(path_or_buf='../salty/data/saltInfo.csv')
df

,cation_SMILES,anion_SMILES,salt_SMILES,cation_name,anion_name,salt_name
0,CCCC[NH+]1C=CN(C1C)C,(C6=C(C5(C)C)C7=CC=CC=C7C=C6)CCCC)(C)C.C(C(F)(...,"CCCC[NH+]1C=CN(C1C)C,(C6=C(C5(C)C)C7=CC=CC=C7C...","1-butyl-2,3-dimethyl-1H-imidazolium",trifluorotris(perfluoroethyl)phosphate(V),"1-butyl-2,3-dimethyl-1H-imidazolium ,trifluoro..."
0,CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-],"CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]",1-ethyl-3-methylimidazolium,trifluoromethanesulfonate,"1-ethyl-3-methylimidazolium,trifluoromethanesu..."
0,CCCCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-],"CCCCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]",1-butyl-3-methylimidazolium,trifluoromethanesulfonate,"1-butyl-3-methylimidazolium ,trifluoromethanes..."
0,CCCCN1C=C[N+](=C1)C,C(=O)(C(F)(F)F)[O-],"CCCCN1C=C[N+](=C1)C,C(=O)(C(F)(F)F)[O-]",1-butyl-3-methylimidazolium,trifluoroacetate,"1-butyl-3-methylimidazolium ,trifluoroacetate"
0,CCN1C=C[N+](=C1)C,C(=C(C#N)C#N)=[N-],"CCN1C=C[N+](=C1)C,C(=C(C#N)C#N)=[N-]",1-ethyl-3-methylimidazolium,tricyanomethanide,"1-ethyl-3-methylimidazolium,tricyanomethanide"
0,CCCCN1C=C[N+](=C1)C,C(=C(C#N)C#N)=[N-],"CCCCN1C=C[N+](=C1)C,C(=C(C#N)C#N)=[N-]",1-butyl-3-methylimidazolium,tricyanomethane,"1-butyl-3-methylimidazolium ,tricyanomethane"
0,CCN1C=C[N+](=C1)C,C(#N)[S-],"CCN1C=C[N+](=C1)C,C(#N)[S-]",1-ethyl-3-methylimidazolium,thiocyanate,"1-ethyl-3-methylimidazolium,thiocyanate"
0,CCCCN1C=C[N+](=C1)C,C(#N)[S-],"CCCCN1C=C[N+](=C1)C,C(#N)[S-]",1-butyl-3-methylimidazolium,thiocyanate,"1-butyl-3-methylimidazolium ,thiocyanate"
0,CCCN1C=C[N+](=C1C)C,[B-](F)(F)(F)F,"CCCN1C=C[N+](=C1C)C,[B-](F)(F)(F)F","1,2-dimethyl-3-propylimidazolium",tetrafluoroborate,"1,2-dimethyl-3-propylimidazolium ,tetrafluorob..."
0,CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F,"CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F",1-ethyl-3-methylimidazolium,tetrafluoroborate,"1-ethyl-3-methylimidazolium,tetrafluoroborate"
